In [1]:
using Flux

┌ Warning: The NVIDIA driver on this system only supports up to CUDA 10.2.0.
│ For performance reasons, it is recommended to upgrade to a driver that supports CUDA 11.2 or higher.
└ @ CUDA C:\Users\anhin\.julia\packages\CUDA\Ozu5O\src\initialization.jl:42


In [2]:
# define ReLU

f(x)=max(0,x) # I don't want to use 'relu', because this would mask the built-in Flux 'relu', 
              #                             and I want to be explicit rather than using a function built into Flux

f (generic function with 1 method)

In [3]:
test_dict = Dict(:x=>0f0, "y"=>4f0, 8=>-3f0)

Dict{Any, Float32} with 3 entries:
  "y" => 4.0
  8   => -3.0
  :x  => 0.0

In [6]:
sum(values(test_dict))

1.0f0

In [10]:
map(f, test_dict) # the root of the problem is that this does not work (and this is horrible!)

LoadError: map is not defined on dictionaries

In [8]:
map(f, values(test_dict))

3-element Vector{Float32}:
 4.0
 0.0
 0.0

In [11]:
map!(f, values(test_dict))

ValueIterator for a Dict{Any, Float32} with 3 entries. Values:
  4.0f0
  0.0f0
  0.0f0

In [12]:
test_dict # let's see if we have modified this dictionary by map! on its values

Dict{Any, Float32} with 3 entries:
  "y" => 4.0
  8   => 0.0
  :x  => 0.0

In [16]:
# we did modify it, and it is rather horrible
# let's perhaps try to encapsulate this awful behavior

function my_map(my_f, my_dict)
    new_dict = deepcopy(my_dict)
    map!(my_f, values(new_dict))
    new_dict
end

my_map (generic function with 1 method)

In [17]:
test_dict = Dict(:x=>0f0, "y"=>4f0, 8=>-3f0)

Dict{Any, Float32} with 3 entries:
  "y" => 4.0
  8   => -3.0
  :x  => 0.0

In [18]:
my_map(f, test_dict)

Dict{Any, Float32} with 3 entries:
  :x  => 0.0
  8   => 0.0
  "y" => 4.0

In [19]:
test_dict

Dict{Any, Float32} with 3 entries:
  "y" => 4.0
  8   => -3.0
  :x  => 0.0

In [20]:
# that was a success; let's see if the gradient just works
p = params(test_dict)

Params([])

In [22]:
sum(values(my_map(f, test_dict)))

4.0f0

In [23]:
grads = gradient(()->sum(values(my_map(f, test_dict))), p)

LoadError: MethodError: no method matching getindex(::Dict{Any, Any})
[0mClosest candidates are:
[0m  getindex(::Dict{K, V}, [91m::Any[39m) where {K, V} at dict.jl:480
[0m  getindex(::AbstractDict, [91m::Any[39m) at abstractdict.jl:494
[0m  getindex(::AbstractDict, [91m::Any[39m, [91m::Any[39m, [91m::Any...[39m) at abstractdict.jl:504

In [24]:
# this is the relevant issue: https://github.com/FluxML/Zygote.jl/issues/408